In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                part_number = html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                src = 'https://ecat.spectrapremium.com/' + html.xpath('//div[@class="part__zoom-gallery"]/descendant::img/@src')[0].strip().replace('/imagettes/', '/').replace('_220.', '.')
                
                # = = = = = = = = = = = = = = =
                
                list_desc = html.xpath('//div[@class="part__nom-spec-valeur-spec"]')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Type': '',
                                         'Vehicle': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])
                
                for desc in list_desc:
                    df_temp.loc[0, desc.xpath('./dt[@class="part__nom-spec"]/text()')[0].strip()] = desc.xpath('./dd[@class="part__valeur-spec"]/text()')[0].strip()

                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：561

https://ecat.spectrapremium.com/en/parts/GMP08B  <->  [ok] - 剩余数量：541 - 当前时间：17:11:46
https://ecat.spectrapremium.com/en/parts/GMP101A  <->  [ok] - 剩余数量：540 - 当前时间：17:11:46
https://ecat.spectrapremium.com/en/parts/GMP07B  <->  [ok] - 剩余数量：539 - 当前时间：17:11:46
https://ecat.spectrapremium.com/en/parts/GMP10A  <->  [ok] - 剩余数量：538 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP05A  <->  [ok] - 剩余数量：537 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP01A  <->  [ok] - 剩余数量：536 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP102A  <->  [ok] - 剩余数量：535 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP07A  <->  [ok] - 剩余数量：534 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP02A  <->  [ok] - 剩余数量：533 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP04A  <->  [ok] - 剩余数量：532 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/GMP109A  <->  [ok] - 剩余数量：531 - 当前时间：17:11:47
https://ecat.spectrapremium.com/en/parts/G

https://ecat.spectrapremium.com/en/parts/FP94A  <->  [ok] - 剩余数量：445 - 当前时间：17:11:58
https://ecat.spectrapremium.com/en/parts/CRP32A  <->  [ok] - 剩余数量：444 - 当前时间：17:11:58
https://ecat.spectrapremium.com/en/parts/CRP29A  <->  [ok] - 剩余数量：443 - 当前时间：17:11:59
https://ecat.spectrapremium.com/en/parts/CRP35A  <->  [ok] - 剩余数量：442 - 当前时间：17:11:59
https://ecat.spectrapremium.com/en/parts/CRP35B  <->  [ok] - 剩余数量：441 - 当前时间：17:11:59
https://ecat.spectrapremium.com/en/parts/CRP34A  <->  [ok] - 剩余数量：440 - 当前时间：17:11:59
https://ecat.spectrapremium.com/en/parts/CRP41A  <->  [ok] - 剩余数量：439 - 当前时间：17:11:59
https://ecat.spectrapremium.com/en/parts/CRP38A  <->  [ok] - 剩余数量：438 - 当前时间：17:11:59
https://ecat.spectrapremium.com/en/parts/CRP43A  <->  [ok] - 剩余数量：437 - 当前时间：17:12:00
https://ecat.spectrapremium.com/en/parts/CRP19B  <->  [ok] - 剩余数量：436 - 当前时间：17:12:00
https://ecat.spectrapremium.com/en/parts/CRP33B  <->  [ok] - 剩余数量：435 - 当前时间：17:12:00
https://ecat.spectrapremium.com/en/parts/CRP45A  <->  [

https://ecat.spectrapremium.com/en/parts/BMP11A  <->  [ok] - 剩余数量：349 - 当前时间：17:12:42
https://ecat.spectrapremium.com/en/parts/BMP13A  <->  [ok] - 剩余数量：348 - 当前时间：17:12:42
https://ecat.spectrapremium.com/en/parts/BMP14A  <->  [ok] - 剩余数量：347 - 当前时间：17:12:42
https://ecat.spectrapremium.com/en/parts/BMP15A  <->  [ok] - 剩余数量：346 - 当前时间：17:12:42
https://ecat.spectrapremium.com/en/parts/BMP16A  <->  [ok] - 剩余数量：345 - 当前时间：17:12:43
https://ecat.spectrapremium.com/en/parts/BMP17A  <->  [ok] - 剩余数量：344 - 当前时间：17:12:43
https://ecat.spectrapremium.com/en/parts/BMP19A  <->  [ok] - 剩余数量：343 - 当前时间：17:12:43
https://ecat.spectrapremium.com/en/parts/BMP20A  <->  [ok] - 剩余数量：342 - 当前时间：17:12:43
https://ecat.spectrapremium.com/en/parts/FP01A  <->  [ok] - 剩余数量：341 - 当前时间：17:12:44
https://ecat.spectrapremium.com/en/parts/CATP05A  <->  [ok] - 剩余数量：340 - 当前时间：17:12:45
https://ecat.spectrapremium.com/en/parts/CRP66A  <->  [ok] - 剩余数量：339 - 当前时间：17:12:45
https://ecat.spectrapremium.com/en/parts/BMP22A  <->  

https://ecat.spectrapremium.com/en/parts/HYP05A  <->  [ok] - 剩余数量：250 - 当前时间：17:13:10
https://ecat.spectrapremium.com/en/parts/HOP20A  <->  [ok] - 剩余数量：249 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MZP03A  <->  [ok] - 剩余数量：248 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MIP13A  <->  [ok] - 剩余数量：247 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/HOP35A  <->  [ok] - 剩余数量：246 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MZP01A  <->  [ok] - 剩余数量：245 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MIP11A  <->  [ok] - 剩余数量：244 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MZP06A  <->  [ok] - 剩余数量：243 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MZP07A  <->  [ok] - 剩余数量：242 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MZP02A  <->  [ok] - 剩余数量：241 - 当前时间：17:13:11
https://ecat.spectrapremium.com/en/parts/MZP11A  <->  [ok] - 剩余数量：240 - 当前时间：17:13:12
https://ecat.spectrapremium.com/en/parts/MZP12A  <->  

https://ecat.spectrapremium.com/en/parts/MIP06A  <->  [ok] - 剩余数量：153 - 当前时间：17:13:23
https://ecat.spectrapremium.com/en/parts/MIP05A  <->  [ok] - 剩余数量：152 - 当前时间：17:13:23
https://ecat.spectrapremium.com/en/parts/MIP08A  <->  [ok] - 剩余数量：151 - 当前时间：17:13:23
https://ecat.spectrapremium.com/en/parts/MIP07A  <->  [ok] - 剩余数量：150 - 当前时间：17:13:24
https://ecat.spectrapremium.com/en/parts/NSP38A  <->  [ok] - 剩余数量：149 - 当前时间：17:13:24
https://ecat.spectrapremium.com/en/parts/NSP39A  <->  [ok] - 剩余数量：148 - 当前时间：17:13:24
https://ecat.spectrapremium.com/en/parts/NSP47A  <->  [ok] - 剩余数量：147 - 当前时间：17:13:25
https://ecat.spectrapremium.com/en/parts/SUP01A  <->  [ok] - 剩余数量：146 - 当前时间：17:13:25
https://ecat.spectrapremium.com/en/parts/NSP40A  <->  [ok] - 剩余数量：145 - 当前时间：17:13:25
https://ecat.spectrapremium.com/en/parts/NSP44A  <->  [ok] - 剩余数量：144 - 当前时间：17:13:25
https://ecat.spectrapremium.com/en/parts/SUP03A  <->  [ok] - 剩余数量：143 - 当前时间：17:13:25
https://ecat.spectrapremium.com/en/parts/OPA001  <->  

https://ecat.spectrapremium.com/en/parts/GMP92A  <->  [ok] - 剩余数量：56 - 当前时间：17:13:38
https://ecat.spectrapremium.com/en/parts/HOP01A  <->  [ok] - 剩余数量：55 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/GMP88A  <->  [ok] - 剩余数量：54 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/GMP98A  <->  [ok] - 剩余数量：53 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/VWP43A  <->  [ok] - 剩余数量：52 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/VWP45A  <->  [ok] - 剩余数量：51 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/GMP84A  <->  [ok] - 剩余数量：50 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/GMP97A  <->  [ok] - 剩余数量：49 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/VWP44A  <->  [ok] - 剩余数量：48 - 当前时间：17:13:39
https://ecat.spectrapremium.com/en/parts/VWP57A  <->  [ok] - 剩余数量：47 - 当前时间：17:13:40
https://ecat.spectrapremium.com/en/parts/GMP83A  <->  [ok] - 剩余数量：46 - 当前时间：17:13:40
https://ecat.spectrapremium.com/en/parts/VWP47A  <->  [ok] - 剩余数量